In [1]:
import matplotlib.pyplot as plt
from mip import *

from src.read_instance import MMURP

In [4]:
instance_folder = "Instancias"
instance_name = "ES-n78-m2-Q10138_with_depot"

filepath = f"{instance_folder}/{instance_name}.txt"

In [5]:
N, D, V, Q, max_dist_nodes, q, c, vehicles_depot, coord_x, coord_y = MMURP(filepath)

Usando a formulação $MDOVRP_{2i− flv}$ de Lalla-Ruiz e Mes (2019)


    Lalla-Ruiz, Eduardo, and Martijn Mes. "Mathematical formulations and improvements for the multi-depot open vehicle routing problem." Optimization Letters 15 (2021): 271-286.

In [6]:
print(D)
print(vehicles_depot)

[76, 77]
[4, 4]


In [7]:
#cria o modelo
model = Model('MMURP', solver_name = GUROBI)

# Variaveis de decisao
x = [[model.add_var(var_type=BINARY) if i!=j else model.add_var(lb=0, ub=0) for i in V] for j in V]
u = [[model.add_var(var_type=CONTINUOUS) if i!=j else model.add_var(lb=0, ub=0) for i in V] for j in V]
y = [model.add_var(var_type=BINARY) for i in N]
w = [model.add_var(var_type=BINARY) for i in N]

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID
Academic license - for non-commercial use only - registered to thiago.giachetto@aluno.ufop.edu.br


\begin{equation}
\label{eq:9}
    x_{ij} \in \{0, 1\}, \quad \forall i, j \in V
\end{equation}

\begin{equation}
\label{eq:10}
    u_{ij} \geq 0, \quad \forall i, j \in V
\end{equation}

\begin{equation}
\label{eq:10.1}
    w_{j} \in \{0, 1\}, \quad \forall j \in N
\end{equation}

In [14]:
def objective(alfa, c, q, x, w):
    A = alfa
    B = 1 - alfa
    
    norm_dist = sum([c[i,j] for i in V for j in V if j>i])
    norm_dem = sum(q)
    
    
    return minimize(A/norm_dist*xsum(x[i][j] * c[i,j] for i in V for j in V if i != j) - \
                    B/norm_dem*xsum(q[j]*w[j] for j in N))

\begin{equation}
\label{eq:1}
 Minimizar \quad A\sum_{i \in V }\sum_{j \in V} c_{ij}x_{ij} - B\sum_{j \in N} q_{j}w_{j}
\end{equation}

In [16]:
# Restricao 3
for j in N:
    model += xsum(x[i][j] for i in V if i != j) == w[j]

\begin{equation}
    \sum_{i \in V, i \neq j}x_{ij} = w_{j}, \quad \forall j \in N
    \tag{2}
\end{equation}

In [15]:
# Restricao 4 - quantidade veiculos por deposito

for pos, k in enumerate(D):
    model += xsum(x[k][i] for i in N) <= vehicles_depot[pos]

In [17]:
# Restricao 5 -> na demanda relacionado ao limite de caminhões

model += xsum(q[j]*w[j] for j in N) <= sum(vehicles_depot)*Q

Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored


In [18]:
# Restricao 6 -> de distancia maxima entre duas cidades

for i in V:
    for j in V:
        if i != j:
            model += c[i,j]*x[i][j] <= max_dist_nodes

In [19]:
# Restricao 7

for j in N:
    model += (xsum(x[i][j] for i in V if i != j) - xsum(x[j][i] for i in N if i != j)) >= 0

\begin{equation}
    \sum_{i \in V, i \neq j}x_{ij} - \sum_{i \in N, i \neq j}x_{ji} \geq 0, \quad \forall j \in N
    \tag {18}
\end{equation}

In [20]:
# Restricao 8
for i in V:
    for j in V:
        if i != j:
            model += x[i][j] + x[j][i] <= 1

\begin{equation}
    x_{ij} + x_{ji} \leq 1, \quad \forall i, j \in V, i \neq j
    \tag {19}
\end{equation}

In [21]:
# Restricao 9
# Ninguem volta para o depósito
model += xsum(x[j][k] for k in D for j in V) == 0

\begin{equation}
    \sum_{k \in D}\sum_{j \in V}x_{jk} = 0
    \tag {20}
\end{equation}

In [22]:
# Restricao 10 -> se cidade não é atendida, restrição é esquecida
for j in N:
    model += (xsum(u[i][j] for i in V if i != j) - xsum(u[j][i] for i in V if i != j) - q[j]) >= -Q*(1 - w[j])

\begin{equation}
    (\sum_{i \in V, i \neq j}u_{ij} - \sum_{i \in V, i \neq j}u_{ji}) - q_{j} \geq -Q(1 - w_{j}), \quad \forall j \in N
    \tag {21}
\end{equation}

    Wolsey, Laurence A. Integer programming (pp. 11). John Wiley & Sons, 2020.

In [23]:
#Restricao 11
for i in N:
    for j in N:
        model += (Q - q[i])*x[i][j] >= u[i][j]

\begin{equation}
    (Q - q_i) \cdot x_{ij} \geq u_{ij}, \quad \forall i, j \in N
    \tag {22}
\end{equation}

In [24]:
# Restricao 12
for k in D:
    for j in N:
        model += Q * x[k][j] >= u[k][j]

\begin{equation}
    Q \cdot x_{kj} \geq u_{kj}, \quad \forall k \in D, j \in N
    \tag {23}
\end{equation}

In [25]:
# # Restricao 13
# menor distancia de algum deposito ate a cidade
d = [min([c[j, i] for j in D]) for i in N]
# menor distancia entre cidade i e quaisquer outra cidade
r = [min([c[j, i] for j in N if i != j]) for i in N]
# máxima menor distancia entre duas cidades
M = max(r)

# d[i] >= r[i] <=> é mais longe ir de algum depósito do q de outra cidade => y in {0, 1}
# d[i] < r[i] <=> é mais perto sair de algum depósito do q de qualquer outra cidade => y = 1
for i in N:
    model += d[i] + M * y[i] >= r[i]*w[i]

\begin{equation}
    d_{i} + M y_{i} \geq r_{i}w_{i}, \quad \forall k \in N
    \tag {8}
\end{equation}

In [26]:
# Restricao 14

for i in N:
    k_l = D[np.argmin([c[j, i] for j in D], axis=0)]
    model += x[k_l][i] >= y[i]

\begin{equation}
    x_{k'i} \geq y_{i}, \quad \forall i \in N, k' = argmin(c_{ki})_{k \in D}
    \tag {9}
\end{equation}

## Calculation of distance and demand of the k-esime feasible solution

In [28]:
dist = lambda x, k: sum([(1 if x[i][j].xi(k)>0.98 else 0) * c[i,j] for i in V for j in V if i != j])
demanda = lambda w, k: sum((q[j] if w[j].xi(k)>0.98 else 0) for j in N)

## Calculating the pareto front

In [ ]:
import numpy as np

steps = 30
solutions_obj = set()

for alfa in np.linspace(0, 1, steps, endpoint=True):
    model.objective = objective(alfa, c, q, x, w)
    num_solutions = 0
    
    for i in range(5):
        model.threads = 2
        status = model.optimize(max_seconds=60)
#         status = OptimizationStatus.FEASIBLE
    
        if status in (OptimizationStatus.INFEASIBLE, OptimizationStatus.ERROR):
            break
            
        if model.num_solutions > num_solutions:
            num_solutions = model.num_solutions
            
            for k in range(model.num_solutions):
                obj_calc = (dist(x,k), demanda(w, k))
                
#                 if obj_calc is not in solutions_obj:
                solutions_obj.add(obj_calc)
                    
        
        if status == OptimizationStatus.OPTIMAL:
            break
        
        print(f"number of solutions: {len(solutions_obj)}")
        break

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (linux64)
Thread count: 2 physical cores, 4 logical processors, using up to 2 threads
Academic license - for non-commercial use only - registered to thiago.giachetto@aluno.ufop.edu.br
Optimize a model with 18324 rows, 12320 columns and 59808 nonzeros
Model fingerprint: 0x5d3d1e55
Variable types: 6162 continuous, 6158 integer (6158 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+04]
  Objective range  [4e-03, 5e-02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 8e+04]

MIP start from previous solve produced solution with objective -0.936502 (0.01s)
Loaded MIP start from previous solve with objective -0.936502

Presolve removed 13870 rows and 6495 columns
Presolve time: 0.11s
Presolved: 4454 rows, 5825 columns, 21761 nonzeros
Variable types: 2832 continuous, 2993 integer (2993 binary)

Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...

Root barrier log...

Ordering time: 0.02s

Barrier

     0     2   -0.93464    0   64   -0.90523   -0.93464  3.25%     -    6s
H   34    36                      -0.9100131   -0.93464  2.71%   459    8s
   192   153   -0.93196    9   37   -0.91001   -0.93464  2.71%   218   10s
   527   437   -0.92867   57   19   -0.91001   -0.93464  2.71%   194   15s
H  727   506                      -0.9116119   -0.93461  2.52%   191   19s
   729   508   -0.93461   13  148   -0.91161   -0.93461  2.52%   190   21s
   742   521   -0.93461   18  126   -0.91161   -0.93461  2.52%   198   25s
H  759   505                      -0.9116169   -0.93461  2.52%   201   27s
   787   567   -0.93461   31  115   -0.91162   -0.93461  2.52%   204   31s
H  833   567                      -0.9116207   -0.93461  2.52%   205   33s
   872   595   -0.92123   37  109   -0.91162   -0.93461  2.52%   206   35s
  1075   969   -0.93461   34  110   -0.91162   -0.93461  2.52%   209   41s
  1662  1084   -0.92601  112   89   -0.91162   -0.93461  2.52%   189   45s
H 1719  1042             

In [ ]:
import numpy as np
def non_dominated(pontos):
    """"
    By Gladston Moreira

    Non-dominate dynamic programming  -  O(n log n)

    Input: 
    pontos = conjunto de pontos 

    Output:
    (idx_non_dominated, idx_dominated) = (indice dos pontos não dominados, indice dos pontos dominados)

    """

    n = pontos.shape[0] # quantidade  de pontos

    dominated = np.zeros(n,dtype=bool)
    # np.lexsort((weight, profit))
    # ip = np.lexsort((pontos[:, 1][::1], pontos[:, 0])) # sort first column 1 and then 2 column
    ip = np.lexsort(pontos[:,::-1].T, axis=-1)
    pontos = pontos[ip] 



    D = True # Decision Variable

    for i in range(n-1):
    # store the time and the last node visited
        if D:
            z = pontos[i,1]

        if  pontos[i+1,1] < z: # i+1 is non-dominated
            D = True
        else:
            dominated[i+1] = True
            D = False

    return (ip[~dominated], ip[dominated])


In [ ]:
first_test = [[sol[0], -sol[1]] for sol in solutions_obj]

idx_non_dominated, idx_dominated = non_dominated(np.array(first_test))

x_plot = np.array([p[0] for p in first_test])
y_plot = np.array([p[1] for p in first_test])

fig, ax = plt.subplots(figsize=(8, 8))
plt.title('Espírito Santo')
plt.xlabel('Distância')
plt.ylabel('Demanda')

ax.scatter(x_plot[idx_non_dominated], y_plot[idx_non_dominated], c='tab:red', label='Non Dominated', alpha=0.5, edgecolors='none')
ax.scatter(x_plot[idx_dominated], y_plot[idx_dominated], c='tab:blue', label='Dominated', alpha=0.5, edgecolors='none')

ax.legend()
# ax.grid(True)

filename_fig = f"{instance_name}_solver_0.png"
plt.savefig(filename_fig)

plt.show()

In [ ]:
def save_result(outfilepath, solutions_obj):
    with open(outfilepath, 'w') as f:
        f.write("distancia, demanda\n")

        for sol in solutions_obj:
            f.write(f"{sol[0]}, {sol[1]}\n")

In [ ]:
# outfilepath = f"{instance_name}.csv"
# save_result(outfilepath, solutions_obj)

In [ ]:
# # Resolve o modelo        
# # model.optimize(max_seconds=7200)
# model.objective = objective(0, c, q, x, w)
model.optimize(max_seconds=60)
print("status: {} objective value : {} best possible: {}".format(model.status, model.objective_value, model.objective_bound))

In [ ]:
# model.gap

In [ ]:
# for i in range(5):
#     status = model.optimize(max_seconds=30)
    
#     if status == OptimizationStatus.OPTIMAL:
#         print("found optimal")
#     elif status == OptimizationStatus.FEASIBLE:
#         print("found feasible")
#     elif status == OptimizationStatus.NO_SOLUTION_FOUND:
#         print("run again")

---

## Testando resultados

In [ ]:
# # dist percorrida
# sum([(1 if x[i][j].x>0.5 else 0) * c[i,j] for i in V for j in V if i != j])

In [ ]:
# # demanda total atendida
# NUM_VEICULOS = sum(vehicles_depot)
# print(f"demanda atendida: {sum(q[j]*w[j].x for j in N)} | limite demanda: {NUM_VEICULOS*Q} | total demanda: {sum(q)}")

In [ ]:
# # --- PLOTA O GRAFICO ---
# plt.figure(figsize=(10, 6))
# plt.scatter(coord_x[0:], coord_y[0:])
# for i in N:
#     plt.annotate(f"{i}", (coord_x[i], coord_y[i]))
# for i in D:
#     plt.plot(coord_x[i], coord_y[i], c = 'r', marker = 's')

# for i in V:
#     for j in V:
#         if x[i][j].x > 0:
#             plt.plot([coord_x[i], coord_x[j]], [coord_y[i], coord_y[j]], c='g', zorder=0)
    
# #escala dos eixos
# plt.yticks([i for i in range(1,80,20)]); 
# plt.xticks([i for i in range(1,80,20)]); 

In [ ]:
# # para onde cada veículo vai
# for k in D:
#     print(f"{k} - ", end='')
#     print([i for i, el in enumerate(u[k]) if el.x > 0])

In [ ]:
# # caminho dos carros olhando arcos
# for k in D:
#     print(f"Depot: {k}")
#     for i, el in enumerate(x[k]):
#         if el.x > 0:
#             visitados = [i]
#             next_list = [j for j, el in enumerate(x[i]) if el.x > 0]
#             while len(next_list) > 0:
#                 if len(next_list) > 1:
#                     print(f"ERROR: {next_list}")
#                 next_el = next_list[0]
#                 visitados.append(next_el)
#                 next_list = [j for j, el in enumerate(x[next_el]) if el.x > 0]
#             print(visitados)

In [ ]:
# # caminho dos carros olhando demanda
# for k in D:
#     print(f"Depot: {k}")
#     for i, el in enumerate(u[k]):
#         if int(el.x) > 0:
#             visitados = [(i, int(el.x))]
#             next_list = [(j, int(el.x)) for j, el in enumerate(u[i]) if int(el.x) > 0]
#             while len(next_list) > 0:
#                 if len(next_list) > 1:
#                     print(f"ERROR: {next_list}")
#                 next_el, value = next_list[0]
#                 visitados.append((next_el, value))
#                 next_list = [(j, int(el.x)) for j, el in enumerate(u[next_el]) if int(el.x) > 0]
#             print([(*el, q[el[0]]) for el in visitados])

In [ ]:
# print("Cidades não atendidas")

# # cidades não atendidas (olhando arestas)
# print(f"aresta:  {([j for j in N if sum([x[i][j].x for i in V]) < 0.5])}")

# # nao atendidos olhando demanda
# print(f"demanda: {[j for j in N if (q[j] - (sum([u[i][j].x for i in V if i != j]) - sum([u[j][i].x for i in N if i != j]))) > 0.05]}")

# # nao atendidos olhando w
# print(f"w:       {[j for j in N if w[j].x < 0.5]}")

In [ ]:
# # nao atendidos olhando w
# for j in N:
#     if w[j].x < 0.5:
#         print(f"{j}-> demanda: {q[j]}")

In [ ]:
# # nao atendidos olhando demanda
# for j in N:
#     enter = (sum([u[i][j].x for i in V if i != j]) - sum([u[j][i].x for i in N if i != j]))
#     if (q[j] - enter) > 0.05:
#         print(f"{j}-> enter: {enter}  need: {q[j]}")

In [ ]:
# # nro de veiculos por depot

# for k in D:
#     print(f"depot {k}: {sum(x[k][i].x  for i in N)} vehicles")

In [ ]:
# # cidades que tem mais de dois arcos ligadas a ela (depositos podem ter mais)
# print([j for j in V if len([i for i in V if x[j][i].x > 0.5 or x[i][j].x > 0.5]) > 2])

In [ ]:
# print(f"distancia media : {sum([c[el] for el in c])/len(c)}")
# print(f"distancia maxima: {max([c[el] for el in c])}")
# print(f"distancia minima: {min([c[el] for el in c])}")